<a href="https://colab.research.google.com/github/coldfir3/FastAI-Demos/blob/main/NFL_HELMET_NN_registration_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
import os
is_colab = 'google.colab' in sys.modules
!nvidia-smi

Mon Aug 23 21:44:32 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
if is_colab:
    !pip install -Uqqq fastai

     |████████████████████████████████| 186 kB 7.4 MB/s 
     |████████████████████████████████| 56 kB 1.0 MB/s 


In [3]:
from fastai.vision.all import *

In [4]:
if is_colab:

    from google.colab import drive
    from os.path import exists

    !gdown --id 1OBNx1-3eTevJnHRZyUKRycA4rGIDXY93
    !gdown --id 1i1_K5ViNap1X0xVczkbyu_PK1MBsGfuX

    # if not exists('/content/drive'):
    #     drive.mount('/content/drive')
    # %run '/content/drive/MyDrive/Kaggle/My Kaggle [colab].ipynb'
    # setup_kaggle()

    # ! kaggle competitions download -c nfl-health-and-safety-helmet-assignment -f train_player_tracking.csv
    # ! kaggle competitions download -c nfl-health-and-safety-helmet-assignment -f train_labels.csv

Downloading...
From: https://drive.google.com/uc?id=1OBNx1-3eTevJnHRZyUKRycA4rGIDXY93
To: /content/train_labels.csv.zip
7.69MB [00:00, 67.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1i1_K5ViNap1X0xVczkbyu_PK1MBsGfuX
To: /content/train_player_tracking.csv.zip
5.80MB [00:00, 91.8MB/s]


## Dataframe loading & processing

In [5]:
# code from: https://www.kaggle.com/robikscube/nfl-helmet-assignment-getting-started-guide
def add_track_features(tracks, fps=59.94, snap_frame=10):
    """
    Add column features helpful for syncing with video data.
    """
    tracks = tracks.copy()
    tracks["game_play"] = (
        tracks["gameKey"].astype("str")
        + "_"
        + tracks["playID"].astype("str").str.zfill(6)
    )
    tracks["time"] = pd.to_datetime(tracks["time"])
    snap_dict = (
        tracks.query('event == "ball_snap"')
        .groupby("game_play")["time"]
        .first()
        .to_dict()
    )
    tracks["snap"] = tracks["game_play"].map(snap_dict)
    tracks["isSnap"] = tracks["snap"] == tracks["time"]
    tracks["team"] = tracks["player"].str[0].replace("H", "Home").replace("V", "Away")
    tracks["snap_offset"] = (tracks["time"] - tracks["snap"]).astype(
        "timedelta64[ms]"
    ) / 1_000
    # Estimated video frame
    tracks["est_frame"] = (
        ((tracks["snap_offset"] * fps) + snap_frame).round().astype("int")
    )
    return tracks

def drop_track_features(tracks):
    tracks = tracks.query("est_frame > 0").reset_index(drop = True)    
    tracks.rename(columns={'player':'label', 'est_frame':'frame'}, inplace = True)
    tracks['view'] = 'Tracking'
    tracks['game_play_frame'] = tracks.apply(lambda x: f"{x['game_play']}_{x['frame']}", axis = 1)
    return tracks[['game_play_frame', 'view', 'x', 'y', 'label']]
    
def add_frame_features(videos):
    videos['game_play'] = videos['video_frame'].apply(lambda x: '_'.join(x.split('_')[:2]))
    videos['x'] = (videos['left'] + videos['width']/2).astype(int).values
    videos['y'] = (videos['top'] + videos['height']/2).astype(int).values
    return videos

def drop_frame_features(videos):
    videos.query("not isSidelinePlayer").reset_index(drop=True)
    videos['game_play_frame'] = videos.apply(lambda x: f"{x['game_play']}_{x['frame']}", axis = 1)
    return videos[['game_play_frame', 'view', 'x', 'y', 'label']]

In [6]:
if is_colab: 
    track_df = pd.read_csv('/content/train_player_tracking.csv.zip')
    frame_df = pd.read_csv('/content/train_labels.csv.zip')
else:
    pass

frame_df = add_frame_features(frame_df)
frame_df = drop_frame_features(frame_df)

track_df = add_track_features(track_df)
track_df = drop_track_features(track_df)

endzone = frame_df.query("view == 'Endzone'").groupby('game_play_frame').apply(lambda x: x[['x', 'y']].values).to_frame('Endzone')
sideline = frame_df.query("view == 'Sideline'").groupby('game_play_frame').apply(lambda x: x[['x', 'y']].values).to_frame('Sideline')
tracking = track_df.groupby('game_play_frame').apply(lambda x: x[['x', 'y']].values).to_frame('Tracking')

df = pd.concat([endzone, sideline, tracking], axis = 1).dropna()

In [7]:
def pad_crop_coord(coord, len = 22):
    l = coord.shape[0]
    if  l < len:
        return np.pad(coord, ((0, len - l), (0,0)))
    elif l > len:
        return coord[:len]
    else:
        return coord

In [8]:
class PadCropPoints(Transform):
    def __init__(self, len = 22): store_attr()

    def encodes(self, x:TensorPoint):
        l = x.shape[0]
        if  l < self.len:
            return F.pad(x, (0, 0, 0, self.len - l))
        elif l > self.len:
            return x[:self.len]
        else:
            return x

In [9]:
class myPointScaler(Transform):
    def __init__(self, mean=None, std=None, axis=1): 
        self.mean = np.array(mean)
        self.std = np.array(std)

    def encodes(self, x:np.ndarray): return (x-self.mean) / self.std
    def encodes(self, x:TensorPoint): return (x-self.mean) / self.std
    def decodes(self, x:TensorPoint):
        f = to_cpu if x.device.type=='cpu' else noop
        return (x*f(self.std) + f(self.mean))

In [10]:
myPointBlock = TransformBlock(type_tfms=TensorPoint.create, item_tfms=PadCropPoints(22))

In [11]:
dblock = DataBlock(blocks=(myPointBlock, myPointBlock, myPointBlock, myPointBlock, myPointBlock),
                   splitter=RandomSplitter(),
                   get_x=(
                        Pipeline([ColReader('Endzone'), myPointScaler((640, 300), (260, 100))]),
                        Pipeline([ColReader('Sideline'), myPointScaler((640, 300), (260, 100))]), 
                        Pipeline([ColReader('Tracking'), myPointScaler((60, 25), (30, 10))])
                   ),
                   get_y=(
                       Pipeline([ColReader('Endzone'), myPointScaler((640, 300), (260, 100))]),
                       Pipeline([ColReader('Sideline'), myPointScaler((640, 300), (260, 100))])
                   ),
                   n_inp = 3
                   )

In [12]:
dls = dblock.dataloaders(df)

In [13]:
x1, x2, x3, y1, y2 = dls.one_batch()

In [14]:
x1.shape, y1.shape

(torch.Size([64, 22, 2]), torch.Size([64, 22, 2]))

## Network

In [15]:
class RegNet(Module):
    def __init__(self, np, n_hid = 256):
        super().__init__()

        self.l_side_in = nn.Linear(2*np, n_hid)
        self.l_side_hid1 = nn.Linear(n_hid, n_hid)

        self.l_end_in = nn.Linear(2*np, n_hid)
        self.l_end_hid1 = nn.Linear(n_hid, n_hid)

        self.l_track_in = nn.Linear(2*np, n_hid)
        self.l_track_hid1 = nn.Linear(n_hid, n_hid)

        self.l_concat_end_in = nn.Linear(3*n_hid, n_hid)
        self.l_concat_end_hid1 = nn.Linear(n_hid, n_hid)
        self.l_concat_end_out = nn.Linear(n_hid, 9)

        self.l_concat_side_in = nn.Linear(3*n_hid, n_hid)
        self.l_concat_side_hid1 = nn.Linear(n_hid, n_hid)
        self.l_concat_side_out = nn.Linear(n_hid, 9)

        self.act = nn.ReLU(inplace=True)


    def get_H(self, p_end, p_side, p_track):

        p_end = p_end.flatten(1, -1)
        p_side = p_side.flatten(1, -1)
        p_track = p_track.flatten(1, -1)

        p_end = self.l_end_in(p_end)
        p_end = self.act(p_end)
        p_end = self.l_end_hid1(p_end)
        p_end = self.act(p_end)

        p_side = self.l_side_in(p_side)
        p_side = self.act(p_side)
        p_side = self.l_side_hid1(p_side)
        p_side = self.act(p_side)

        p_track = self.l_track_in(p_track)
        p_end = self.act(p_end)
        p_track = self.l_track_hid1(p_track)
        p_end = self.act(p_end)

        p_concat = torch.cat([p_end, p_side, p_track], dim = -1)

        pred_end = self.l_concat_end_in(p_concat)
        pred_end = self.act(pred_end)
        pred_end = self.l_concat_end_hid1(pred_end)
        pred_end = self.act(pred_end)
        pred_end = self.l_concat_end_out(pred_end)

        pred_side = self.l_concat_side_in(p_concat)
        pred_side = self.act(pred_side)
        pred_side = self.l_concat_side_hid1(pred_side)
        pred_side = self.act(pred_side)
        pred_side = self.l_concat_side_out(pred_side)

        return pred_end.view(-1, 3, 3), pred_side.view(-1, 3, 3)

    def forward(self, p_end, p_side, p_track):

        H_end, H_side = self.get_H(p_end, p_side, p_track)

        p_track = F.pad(p_track, (0, 1, 0, 0), value = 1.)
        pred_end = torch.matmul(p_track, H_end)[:,:,:-1]
        pred_side = torch.matmul(p_track, H_end)[:,:,:-1]

        return torch.cat((pred_end, pred_side), dim = -2)



In [16]:
model = RegNet(22)

In [17]:
model(x1.cpu(), x2.cpu(), x3.cpu()).shape

torch.Size([64, 44, 2])

## Loss

In [18]:
# def min_mse(preds, t1, t2):
#     targets = (t1, t2)
#     loss = 0
#     for p, t in zip(preds, targets):
#         t = t.contiguous()
#         d = torch.cdist(p, t)
#         loss = loss + (d.min(dim = 1).values**2).mean().sqrt()
#     # d = torch.cdist(preds[0], targets[0])
#     # loss = (d.min(dim = 1).values**2).mean().sqrt()

#     return loss

In [19]:
class stack_y(Callback):
    def before_batch(self):
        yb = torch.cat(self.learn.yb, dim = -2)
        # yb = F.pad(yb, (0, 1, 0, 0), value = 1.)
        self.learn.yb = tuple([yb,])
        # self.learn.yb = tuple([self.learn.yb[0],])


In [20]:
learn = Learner(dls, model, loss_func = MSELossFlat(), cbs = stack_y())

In [21]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,time


RuntimeError: ignored